# Parsing Quarterly Financials

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from tqdm import tqdm

In [4]:
datadir = 'Income_SP500/'
filelist = os.listdir(datadir)

In [5]:
feats=['TotalRevenue', 'OperatingRevenue', 'PretaxIncome', 'NetIncomeCommonStockholders', 'NetIncome', 'NetIncomeIncludingNoncontrollingInterests', 'NetIncomeContinuousOperations', 'DilutedNIAvailtoComStockholders', 'NetIncomeFromContinuingAndDiscontinuedOperation', 'NormalizedIncome', 'ReconciledDepreciation', 'NetIncomeFromContinuingOperationNetMinorityInterest', 'TaxRateForCalcs', 'TaxEffectOfUnusualItems']
companies=[]
data=np.empty((len(filelist),len(feats)),dtype=object)
for it,filename in enumerate(filelist):
    companies.append(filename.split('_')[0])
    with open(datadir+filename, 'r') as cfile:
        for line in cfile:
            line=[num.replace(',','') for num in line.strip().split('\"') if num.replace(',','') != '']
            if line[0] in feats:
                data[it][feats.index(line[0])]=np.array([float(num) for num in line[1:]])

In [26]:
# print(data[0])
print(data[0].shape)
print(companies[4])

(14,)
KIM


In [28]:
for i in range(14):
    print(len(data[4][i]))

print(feats[2])
print(data[4][2])
print(feats[10])
print(data[4][10])

123
123
85
123
123
123
123
123
123
123
90
123
123
123
PretaxIncome
[ 7.45126e+08  3.89420e+07  4.87954e+08  9.70360e+07  1.21194e+08
  1.88248e+08 -5.94290e+07  7.33229e+08  6.58260e+07  9.37860e+07
  6.06490e+07  6.52410e+07  9.23300e+07  2.47398e+08  5.05820e+07
  4.11920e+07  6.01740e+07  2.57060e+07  5.19350e+07  2.12600e+07
  4.74280e+07  3.09990e+07 -1.17880e+07  2.17600e+06  4.66380e+07
  2.98820e+07  2.77080e+07  6.95700e+07  4.17560e+07  3.68820e+07
  3.76660e+07 -4.45320e+07  2.74900e+07  5.15650e+07 -4.03080e+07
 -4.80000e+05  2.40770e+07  7.81000e+05  1.80710e+07  1.15700e+06
  9.26800e+06  2.11580e+07  1.26450e+07  2.78220e+07  1.73870e+07
  1.75830e+07  1.81290e+07  2.21690e+07  3.22470e+07  1.47740e+07
  3.29820e+07 -2.88570e+07  2.02630e+07  3.08140e+07  1.72830e+07
  3.08310e+07  5.02350e+07  1.04400e+06  2.28050e+07  2.65410e+07
  2.76680e+07  3.04680e+07  3.40920e+07  4.65600e+07  5.38690e+07
  5.28810e+07  4.00740e+07  6.38420e+07  5.88580e+07  4.97160e+07
  5.85720

In [4]:
def parse_financials(filename, keys):
    datadict = {}
    with open(filename, 'r') as cfile:
        for line in cfile:
            line = line.strip()
            for key in keys:
                if key in line:
                    data = line.split(',')[1:]
                    data = [float(el.replace(',','')) for el in data]
                    datadict[key] = data
            
            
            filtered = []
            for el in line:
                if el == ',' or el=='':
                    continue
                filtered.append(float(el.replace(',','')))
        print(filtered)


file_i = "Income_SP500/MSFT_quarterly_financials.csv"
parse_financials(file_i)

[184903000000.0, 51728000000.0, 45317000000.0, 46152000000.0, 41706000000.0, 43076000000.0, 37154000000.0, 38033000000.0, 35021000000.0, 36906000000.0, 33055000000.0, 33717000000.0, 30571000000.0, 32471000000.0, 29084000000.0, 30085000000.0, 26819000000.0, 28918000000.0, 24538000000.0, 23317000000.0, 22090000000.0, 24090000000.0, 20453000000.0, 20614000000.0, 20531000000.0, 23796000000.0, 20379000000.0, 22180000000.0, 21729000000.0, 26470000000.0, 23201000000.0, 23382000000.0, 20403000000.0, 24519000000.0, 18529000000.0, 19896000000.0, 20489000000.0, 21456000000.0, 16008000000.0, 18059000000.0, 17407000000.0, 20885000000.0, 17372000000.0, 17367000000.0, 16428000000.0, 19953000000.0, 16195000000.0, 16039000000.0, 14503000000.0, 19022000000.0, 12920000000.0, 13099000000.0, 13648000000.0, 16629000000.0, 15061000000.0, 15837000000.0, 14454000000.0, 16367000000.0, 13762000000.0, 13371000000.0, 14398000000.0, 12542000000.0, 10811000000.0, 11804000000.0, 10900000000.0, 11837000000.0, 97410000

In [12]:
def get_labels():
    datadir = 'Income_SP500/'
    filelist = os.listdir(datadir)
    labels = {}
    for file_i in tqdm(filelist):
        with open(datadir+file_i, 'r') as cfile:
            cfile.readline()
            for line in cfile:
                line = line.strip()
                label = line.split(',')[0]
                # print(line.split(','))
                # print(label)
                if label in labels.keys():
                    labels[label] += 1
                else:
                    labels[label] = 1

        # print(labels)
    return labels

labels = get_labels()
labels = sorted(labels.items(), key=lambda x:x[1], reverse=True)
print(labels)
keys = [item[0] for item in labels[:14]]
print(keys)
n = 0
cut = 503
labels = dict(labels)
for item in labels.items():
    if item[1]>=cut:
        n+=1
print(n)

100%|██████████| 503/503 [00:00<00:00, 1117.72it/s]

[('TotalRevenue', 503), ('OperatingRevenue', 503), ('PretaxIncome', 503), ('NetIncomeCommonStockholders', 503), ('NetIncome', 503), ('NetIncomeIncludingNoncontrollingInterests', 503), ('NetIncomeContinuousOperations', 503), ('DilutedNIAvailtoComStockholders', 503), ('NetIncomeFromContinuingAndDiscontinuedOperation', 503), ('NormalizedIncome', 503), ('ReconciledDepreciation', 503), ('NetIncomeFromContinuingOperationNetMinorityInterest', 503), ('TaxRateForCalcs', 503), ('TaxEffectOfUnusualItems', 503), ('BasicEPS', 501), ('DilutedEPS', 501), ('BasicAverageShares', 501), ('DilutedAverageShares', 501), ('TaxProvision', 500), ('NetInterestIncome', 499), ('InterestExpense', 496), ('TotalUnusualItemsExcludingGoodwill', 494), ('TotalUnusualItems', 494), ('TotalExpenses', 474), ('EBIT', 474), ('OtherIncomeExpense', 473), ('NetNonOperatingInterestIncomeExpense', 470), ('InterestExpenseNonOperating', 466), ('SpecialIncomeCharges', 464), ('OperatingIncome', 455), ('OperatingExpense', 452), ('EBITD